# GraphCast

## 1. 模型简介
Graphcast是谷歌Deepmind提出的一种用于全球中期天气预报的的方法。并支持包括预测热带气旋路径、大气河流和极端气温在内的应用。
该模型将地球天气的两个最新状态(当前时间和六个小时前)作为输入，并预测六小时后的天气状态。一个单独的天气状态由一个0.25维度/经度网格(721x1440)表示，这相当于赤道上大约28x28公里的分辨率，其中每个网格点代表一组地面和大气变量。Graphcast采用自回归的方式预测，可以通过反馈自己的预测结果作为输入来推理，从而生成任意长的天气状态轨迹。



**模型结构**

Graphcast是一种基于机器学习的天气预报大模型，性能优于世界上最准确的机器学习天气预报系统。Deepmind以编码-处理-解码的方式使用图神经网络(Graph Neural Network，GNN)来创建一个自回归模型，共有3760万个参数。
Graphcast三阶段模拟过程如下：

1）第一阶段：编码阶段基于单个GNN将输入网格上表示为节点属性的变量映射到多网格图表示上的学习节点。

2）第二阶段：处理器阶段使用16个非共享GNN层来在多网格上执行学习信息传递，使得能够以很少的消息传递步骤进行有效的本地和远程信息传递。

3）第三阶段：解码器将来自多网格表示的第二阶段最终学习到的特征映射回纬度-经度网格，与第一阶段类似使用单个GNN层，并将输出预测作为残差更新最新的输入状态。

![img](./arch.jpg)



## 2. 软件环境准备

### 2.1 基于dtk适配软件
<p>1. 基础软件环境DTK：推荐环境 dtk=dtk-24.04.2  下载链接：<a href="https://cancon.hpccube.com:65024/1/main/DTK-24.04.2/Ubuntu22.04" target="_blank">https://cancon.hpccube.com:65024/1/main/DTK-24.04.2/Ubuntu22.04 </a> </p>
<p>2. Jax软件包下载：推荐环境 Jax=0.4.23 Jaxlib=0.4.23, py310 下载链接: <a href="https://download.sourcefind.cn:65024/4/main/jax/DAS1.2" target="_blank">https://download.sourcefind.cn:65024/4/main/jax/DAS1.2 </a> </p>

### 2.2 软件环境检查



In [1]:
# 检查jax版本
import jax
import os
version = jax.__version__
num = float(version[:3])
# assert num == 1.10

# 检查硬件环境
device = "cpu"
if os.system('rocm-smi 2>/dev/null || hy-smi 2>/dev/null')==0:
    device = "dtk"

elif os.system('nvidia-smi 2>/dev/null')==0: 
    device = "cuda"

print("jax version:", version)
print("device =", device)

2025-11-26 14:29:29.321176: E external/xla/xla/stream_executor/plugin_registry.cc:90] Invalid plugin kind specified: DNN



============================ System Management Interface =============================
DCU     Temp     AvgPwr     Perf     PwrCap     VRAM%      DCU%      Mode     
0       52.0C    92.0W      manual   300.0W     0%         0%        Normal   
=================================== End of SMI Log ===================================

jax version: 0.4.23
device = dtk


### 2.2 依赖安装


In [2]:
# 依赖安装(镜像中环境已配置，此步骤省略)
!pip install -r requirements.txt --constraint constraints.txt
!pip install -e .
!pip uninstall shapely --yes
!pip install shapely

!pip install --upgrade google-api-python-client
!pip install google.cloud.bigquery
!pip install google.cloud.storage

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Obtaining file:///public/home/esmalls/apprepo/model/20251124150916/graphcast_jax/graphcast_jax
  Preparing metadata (setup.py) ... done
  Attempting uninstall: graphcast
    Found existing installation: graphcast 0.1
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/public/home/shizhm2050/AIWeather/graphcast_jax'


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Found existing installation: shapely 2.1.2
Uninstalling shapely-2.1.2:
  Successfully uninstalled shapely-2.1.2
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua

## 3. 素材准备
### 3.1 数据集准备


推理数据包括两部分，era5数据和标准化数据，推理时可根据模型的分辨率和压力层级数量选择，请确保当前项目中包含dataset和stats目录且结构如下：
```
dataset\
├── source-era5_date-2022-01-01_res-1.0_levels-13_steps-01.nc
├── source-era5_date-2022-01-01_res-0.25_levels-13_steps-01.nc
└── source-era5_date-2022-01-01_res-0.25_levels-37_steps-01.nc

stats\
├── stats_diffs_stddev_by_level.nc
├── stats_mean_by_level.nc
└── stats_stddev_by_level.nc
```

### 3.2 推理模型准备


模型文件包括三个官方开源的预训练模型，具有不同的分辨率和压力层级，请确保当前项目中包含param目录且结构如下：
```
param\
├── GraphCast_small_resolution_1.0_pressure_levels_13.npz
├── GraphCast_operational_resolution_0.25_pressure_levels_13.npz
└── GraphCast_resolution_0.25_pressure_levels_37.npz
```

主要目录结构如下
```
graphcast-jax\
|----graphcast\
|    |----autoregressive.py
|    |----graphcast.py
|    |----casting.py
|    |----data_utils.py
|    |----losses.py
|    |----normalization.py
|    |----xarray_jax.py
|    |----predictor_base.py
|    |----...
|----dataset\
|    |----source-era5_date-2022-01-01_res-1.0_levels-13_steps-01.nc
|    |----source-era5_date-2022-01-01_res-0.25_levels-13_steps-01.nc
|    |----source-era5_date-2022-01-01_res-0.25_levels-37_steps-01.nc
|----stats\
|    |----stats_diffs_stddev_by_level.nc
|    |----stats_mean_by_level.nc
|    |----stats_stddev_by_level.nc
|----param\
|    |----GraphCast_small_resolution_1.0_pressure_levels_13.npz
|    |----mGraphCast_operational_resolution_0.25_pressure_levels_13.npz
|    |----GraphCast_resolution_0.25_pressure_levels_37.npz
|----README.md
|----inference.py
|----run_infer.sh
|----graphcast-jax.ipynb
|----graphcast_official.ipynb
|----constraints.txt
|----requirements.txt
|----setup.py
```

## 4 推理
### 4.1 开始推理
GraphCast官方提供三个预训练模型，可通过修改如下配置进行选择相应的模型和数据执行推理，参数解释如下：
- pretrained：设置进行推理的预训练模型;
- dataset：设置推理的数据文件;
- mode：设置模型参数初始化方式;
- var：设置可视化气象变量，结果图像保存在当前目录;
- level: 可视化大气变量的压力层级设置。

In [3]:
!python inference.py \
    --pretrained "./param/GraphCast_resolution_0.25_pressure_levels_37.npz" \
    --dataset "./dataset/source-era5_date-2022-01-01_res-0.25_levels-37_steps-01.nc" \
    --mode "Checkpoint" \
    --var "temperature" \
    --level 500

2025-11-26 14:30:14.284671: E external/xla/xla/stream_executor/plugin_registry.cc:90] Invalid plugin kind specified: DNN
-----------  Running Arguments -----------
dataset: ./dataset/source-era5_date-2022-01-01_res-0.25_levels-37_steps-01.nc
level: 500
mode: Checkpoint
pretrained: ./param/GraphCast_resolution_0.25_pressure_levels_37.npz
var: temperature
------------------------------------------
Model description:
 
GraphCast model at 0.25deg resolution, with 37 pressure levels. This model is
trained on ERA5 data from 1979 to 2017, and can be causally evaluated on 2018
and later years. This model takes as inputs `total_precipitation_6hr`. This was
described in the paper
`GraphCast: Learning skillful medium-range global weather forecasting`
(https://arxiv.org/abs/2212.12794).
 

Model license:
 
The model weights are licensed under the Creative Commons
Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0). You
may obtain a copy of the License at:
https://creativecommo

## 5. 相关文献和引用
https://github.com/google-deepmind/graphcast
